# Initialisation
Here, we will remove first n volume, and copy data in result directory

In [ ]:
import glob
import os
import subprocess

dummy_scan = 10  #number of volumes for delete

path = '/mnt/d/Data/Pigeon/analysis/ColorDisc/' 
bold_files = glob.glob('%s/sub*/*/func.nii.gz'%(path))



for cur_bold in bold_files:
    print(cur_bold)
    #cur_dir = os.path.dirname(cur_bold)
      # strip off .nii.gz from file name (makes code below easier)
    results_folder = cur_bold[:-7]    
    #print(results_folder)
    
    # creat a result folder fro each run
    os.makedirs(results_folder)
    
    #copy functional data in result folder
    os.system("fslmaths %s %s/prefiltered_func_data -odt float"%(cur_bold,results_folder))
    
    # read number of volumes for each run and remove first five volume
    nVloume = subprocess.check_output("fslnvols %s"%(cur_bold), shell=True).decode('utf-8')
    print(nVloume)
    #remove the dumy scns from data
    os.system("fslroi %s/prefiltered_func_data %s/prefiltered_func_data %s %s"%(results_folder,results_folder,dummy_scan,int(nVloume)-int(dummy_scan)))
    #creat an example file
    os.system("fslroi %s/prefiltered_func_data %s/example_func %s %s"%(results_folder,results_folder,round((int(nVloume)-int(dummy_scan))/2),1))

   

# Registration

Anatomical 2 Standard space registration

In [ ]:
import glob
import os

data_path = '/mnt/d/Data/Pigeon/analysis/ColorDisc/' 
reference_path = '/mnt/d/Data/Pigeon/analysis/ColorDisc/template'
#bold_dir = glob.glob('%s/sub*/run*'%(path))
sub_dir = glob.glob('%s/sub*'%(data_path))

for sub in sub_dir:
    
    reg_directory = os.path.join(sub,'anat/reg')
    
    print(reg_directory)
    os.makedirs(reg_directory)
    
    os.system('fslmaths %s/anat/anat_brain %s/highres'%(sub,reg_directory))
    os.system('fslmaths %s/anat/anat %s/highres_head'%(sub,reg_directory))
    os.system('fslmaths %s/MYtemplate0 %s/standard'%(reference_path,reg_directory))
    os.system('fslmaths %s/MYtemplate0 %s/standard_head'%(reference_path,reg_directory))
    os.system('fslmaths %s/MYtemplate0 -bin -dilF -dilF %s/standard_mask -odt char'%(reference_path,reg_directory))

    # change directory to the reg 
    os.chdir(reg_directory)
    
    #hihres 2 standard
    
    # FSL nonlinear registeration
    os.system('flirt -in highres -ref standard -out highres2standard -omat highres2standard.mat -cost corratio -dof 12 -searchrx -180 180 -searchry -180 180 -searchrz -180 180 -interp trilinear')
    os.system('fnirt --iout=highres2standard_head --in=highres_head --aff=highres2standard.mat --cout=highres2standard_warp --iout=highres2standard --jout=highres2highres_jac --config=T1_2_MNI152_2mm --ref=standard_head --refmask=standard_mask --warpres=10,10,10')
    os.system('applywarp -i highres -r standard -o highres2standard -w highres2standard_warp')
    os.system('convert_xfm -inverse -omat standard2highres.mat highres2standard.mat')      
    os.system('slicer highres2standard standard -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png highres2standard1.png ; slicer standard highres2standard -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png highres2standard2.png ; pngappend highres2standard1.png - highres2standard2.png highres2standard.png; rm -f sl?.png highres2standard2.png')
    os.system('rm highres2standard1.png')

# Plot Anatomical 2 Atlas registration reults

In [ ]:
%matplotlib inline
%pylab inline
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg

from IPython.display import display, Image
import glob
import os

data_path = '/mnt/d/Data/Pigeon/analysis/ColorDisc/' 
reg_dir = glob.glob('%s/sub*/anat/reg'%(data_path))
for curr_reg in reg_dir:
    
    #print(curr_reg)
    print(curr_reg[-9:-4])
    plt.show()
    display(Image(filename=os.path.join(curr_reg, 'highres2standard.png' )))

# Functional 2 anatomical space registration
<br>
ANt is working better than FSL nonlinear registration.
Dow you will find both methods.

In [ ]:
import glob
import os

data_path = '/mnt/d/Data/Pigeon/analysis/ColorDisc/'
func_dir = glob.glob('%s/sub*/*/func'%(data_path))

for curr_func_dir in func_dir[14:15]:
    
    reg_dir = curr_func_dir[0:43]+'/anat/reg'
    print(curr_func_dir)
    print(reg_dir)
    extended_func_dir = curr_func_dir[:-5]

    #copy reg directory to each run directory
    os.system('cp -R %s %s'%(reg_dir,curr_func_dir))
    os.system('cp %s/example_func.nii.gz %s/reg'%(curr_func_dir,curr_func_dir))

    # change path
    os.chdir(curr_func_dir + '/reg')

    #FSL
    # register func 2 extended func
    os.system('fslmaths %s/expanded_func %s/reg/initial_highres'%(extended_func_dir,curr_func_dir))
    os.system('flirt -in example_func -ref initial_highres -out example_func2initial_highres -omat example_func2initial_highres.mat -cost corratio -dof 6 -searchrx -180 180 -searchry -180 180 -searchrz -180 180 -interp trilinear')
    os.system('convert_xfm -inverse -omat initial_highres2example_func.mat example_func2initial_highres.mat')
    os.system('slicer example_func2initial_highres initial_highres -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; /usr/local/fsl/bin/pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png example_func2initial_highres1.png ; /usr/local/fsl/bin/slicer initial_highres example_func2initial_highres -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; /usr/local/fsl/bin/pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png example_func2initial_highres2.png ; /usr/local/fsl/bin/pngappend example_func2initial_highres1.png - example_func2initial_highres2.png example_func2initial_highres.png; /bin/rm -f sl?.png example_func2initial_highres2.png')
    os.system('rm example_func2initial_highres1.png')
    # initial_highres to highres
    os.system('flirt -in initial_highres -ref highres -out initial_highres2highres -omat initial_highres2highres.mat -cost corratio -dof 3 -searchrx -180 180 -searchry -180 180 -searchrz -180 180 -interp trilinear') 
    os.system('convert_xfm -inverse -omat highres2initial_highres.mat initial_highres2highres.mat')
    os.system('slicer initial_highres2highres highres -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; /usr/local/fsl/bin/pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png initial_highres2highres1.png ; /usr/local/fsl/bin/slicer highres initial_highres2highres -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; /usr/local/fsl/bin/pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png initial_highres2highres2.png ; /usr/local/fsl/bin/pngappend initial_highres2highres1.png - initial_highres2highres2.png initial_highres2highres.png; /bin/rm -f sl?.png initial_highres2highres2.png')
    os.system('rm initial_highres2highres1.png')
    # example_func 2 highres
    os.system('convert_xfm -omat example_func2highres.mat -concat initial_highres2highres.mat example_func2initial_highres.mat')
    os.system('flirt -ref highres -in example_func -out example_func2highres -applyxfm -init example_func2highres.mat -interp trilinear')
    os.system('convert_xfm -inverse -omat highres2example_func.mat example_func2highres.mat')
    os.system('slicer example_func2highres highres -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; /usr/local/fsl/bin/pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png example_func2highres1.png ; /usr/local/fsl/bin/slicer highres example_func2highres -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; /usr/local/fsl/bin/pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png example_func2highres2.png ; /usr/local/fsl/bin/pngappend example_func2highres1.png - example_func2highres2.png example_func2highres.png; /bin/rm -f sl?.png example_func2highres2.png')
    os.system('rm example_func2highres1.png')
    #example_func 2 standard
    os.system('convert_xfm -omat example_func2standard.mat -concat highres2standard.mat example_func2highres.mat')
    os.system('convertwarp --ref=standard --premat=example_func2highres.mat --warp1=highres2standard_warp --out=example_func2standard_warp')
    os.system('applywarp --ref=standard --in=example_func --out=example_func2standard --warp=example_func2standard_warp')
    os.system('convert_xfm -inverse -omat standard2example_func.mat example_func2standard.mat')
    os.system('slicer example_func2standard standard -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; /usr/local/fsl/bin/pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png example_func2standard1.png ; /usr/local/fsl/bin/slicer standard example_func2standard -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; /usr/local/fsl/bin/pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png example_func2standard2.png ; /usr/local/fsl/bin/pngappend example_func2standard1.png - example_func2standard2.png example_func2standard.png; /bin/rm -f sl?.png example_func2standard2.png')

# Plot registration results

In [ ]:
%matplotlib inline
%pylab inline
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg

from IPython.display import display, Image
import glob
import os

data_path = '/mnt/d/Data/Pigeon/analysis/ColorDisc/'
reg_dir = glob.glob('%s/sub*/*/func/reg'%(data_path))
for curr_img in reg_dir[14:15]:
    
    print(curr_img)
    #img = mpimg.imread(os.path.join(reg_dir, 'example_func2standard.png' ))
    #imgplot = plt.imshow(img)
    plt.show()
    display(Image(filename=os.path.join(curr_img, 'example_func2standard.png' )))


# Preprocessing

In [ ]:
import glob
import os
import subprocess

data_path = '/mnt/d/Data/Pigeon/analysis/ColorDisc/'
func_dir = glob.glob('%s/sub*/*/func'%(data_path))

MotionCorrection = True
    
SliceTimeCorrection = True
if SliceTimeCorrection:
    TR = 4 #second
    acquisitionType = 'odd'

SpatialSmoothing = True
if SpatialSmoothing:
    smooth = 8 #mm
    
TemporalFiltering = True
if TemporalFiltering:
    paradigm_hp = 80 #second

for curr_bold in func_dir:

    print(curr_bold)
    # change directory to the reg 
    os.chdir(curr_bold)
    
    #motion correction
    if MotionCorrection:
        print('motion correction is running ...')
        os.system("mcflirt -in prefiltered_func_data -out prefiltered_func_data_mcf -mats -plots -reffile example_func -rmsrel -rmsabs -spline_final")
        os.system("mkdir -p mc ; mv -f prefiltered_func_data_mcf.mat prefiltered_func_data_mcf.par prefiltered_func_data_mcf_abs.rms prefiltered_func_data_mcf_abs_mean.rms prefiltered_func_data_mcf_rel.rms prefiltered_func_data_mcf_rel_mean.rms mc")
        os.system("fsl_tsplot -i %s/prefiltered_func_data_mcf.par -t 'MCFLIRT estimated rotations (radians)' -u 1 --start=1 --finish=3 -a x,y,z -w 640 -h 144 -o %s/rot.png"%('mc','mc'))
        os.system("fsl_tsplot -i %s/prefiltered_func_data_mcf.par -t 'MCFLIRT estimated translations (mm)' -u 1 --start=4 --finish=6 -a x,y,z -w 640 -h 144 -o %s/trans.png"%('mc','mc'))
        os.system("fsl_tsplot -i %s/prefiltered_func_data_mcf_abs.rms,prefiltered_func_data_mcf_rel.rms -t 'MCFLIRT estimated mean displacement (mm)' -u 1 -w 640 -h 144 -a absolute,relative -o %s/disp.png"%('mc','mc')) 
    else:
        print('no motion correction')
        os.system("fslmaths prefiltered_func_data prefiltered_func_data_mcf -odt float")         
    
    #slice time correction
    if SliceTimeCorrection:
        print('slice time correction is running ...')
        os.system("slicetimer -i prefiltered_func_data_mcf --out=prefiltered_func_data_st -r %s --%s"%(TR,acquisitionType))
        os.system("fslmaths prefiltered_func_data_st -Tmean mean_func")
    else:
        print('no slice time correction')
        os.system("fslmaths prefiltered_func_data_mcf prefiltered_func_data_st -odt float")
        os.system("fslmaths prefiltered_func_data_st -Tmean mean_func")
        
    # brain extraction         
    os.system("bet2 mean_func mask -f 0.3 -n -m; immv mask_mask mask")
    os.system("fslmaths prefiltered_func_data_st -mas mask prefiltered_func_data_bet")
    
    #spatial smootiong
    if SpatialSmoothing:
        print('spatial smooting is running ...')
        output = subprocess.check_output("fslstats prefiltered_func_data_bet -p 2 -p 98", shell=True).decode('utf-8')
        output_split = output.split()
        int2 = float(output_split[0])
        int98 = float(output_split[1])
        print(int2,int98)#0.000000 954.826904
        brain_thresh =10
        intensity_threshold = int2 + ( brain_thresh * ( int98 - int2 ) / 100.0)
        os.system("fslmaths prefiltered_func_data_bet -thr %s -Tmin -bin mask -odt char"%(intensity_threshold))
        median_intensity = subprocess.check_output("fslstats prefiltered_func_data_st -k mask -p 50", shell=True).decode('utf-8')
        median_intensity = float(median_intensity)
        print(median_intensity)#705.560059 
        os.system("fslmaths mask -dilF mask")
        os.system("fslmaths prefiltered_func_data_st -mas mask prefiltered_func_data_thresh")
        os.system("fslmaths prefiltered_func_data_thresh -Tmean mean_func")
        susan_int = (median_intensity - int2) * 0.75
        smoothsigma = smooth / 2.355 
        os.system("susan prefiltered_func_data_thresh %s %s 3 1 1 mean_func %s prefiltered_func_data_smooth"%(susan_int,smoothsigma,susan_int))
    else:
        print('no spatial smoothing')
        os.system("fslmaths prefiltered_func_data_bet prefiltered_func_data_smooth -odt float")
            
    #Normalization
    normmean = 10000
    scaling = normmean / median_intensity
    os.system("fslmaths prefiltered_func_data_smooth -mas mask prefiltered_func_data_smooth")
    os.system("fslmaths prefiltered_func_data_smooth -mul %s prefiltered_func_data_intnorm"%(scaling))
    
    # temporal filtering
    if TemporalFiltering:
        print('temporal filtering is running...')
        #highpass temporal filtering (Gaussian-weighted least-squares straight line fitting, with sigma=hp_sigma_sec)
        os.system("fslmaths prefiltered_func_data_intnorm -Tmean tempMean")
        hp_sigma_sec = paradigm_hp / 2.0 
        hp_sigma_vol = hp_sigma_sec / TR
        lp_sigma_vol = -1
        
        # if you want to do lowpass filtering, you cn using these lines:
        #Gaussian lowpass temporal filtering, with sigma=lp_sigma_sec
        #lp_sigma_sec = 2.8
        #lp_sigma_vol = lp_sigma_sec / TR
        os.system("fslmaths prefiltered_func_data_intnorm -bptf %s %s -add tempMean prefiltered_func_data_tempfilt"%(hp_sigma_vol, lp_sigma_vol))
        os.system("imrm tempMean")
        os.system("fslmaths prefiltered_func_data_tempfilt filtered_func_data")
    else:
        print('no temporal filtering')
        os.system("fslmaths prefiltered_func_data_intnorm filtered_func_data -odt float")
                  

    os.system("fslmaths filtered_func_data -Tmean mean_func")
    os.system("rm -rf prefiltered_func_data*")

            


# Plot Estimated Motion Parameters

In [ ]:
%matplotlib inline
%pylab inline
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg

from IPython.display import display, Image
import glob
import os

data_path = '/mnt/d/Data/Pigeon/analysis/ColorDisc/'
func_dir = glob.glob('%s/sub*/*/func/mc'%(data_path))
for mc_dir in func_dir:
    
    #print(reg_dir)
    #img = mpimg.imread(os.path.join(reg_dir, 'example_func2standard.png' ))
    #imgplot = plt.imshow(img)
    plt.show()
    display(Image(filename=os.path.join(mc_dir, 'rot.png' )))
    display(Image(filename=os.path.join(mc_dir, 'trans.png' )))